# TensorFlow

TensorFlow는 수학 계산과 데이터의 흐름을 노드(Node)와 엣지(Edge)를 사용한 방향 그래프(Directed Graph)로 표현한다.
노드는 수학적 계산, 데이터 입/출력, 그리고 데이터의 읽기/저장 등의 작업을 수행한다. 엣지는 노드들 간 데이터의 입출력 관계를 나타낸다.
엣지는 동적 사이즈의 다차원 데이터 배열(=Tensor)을 실어나르는데, 여기에서 텐서플로우라는 이름이 지어졌다.

# Terminology

**오퍼레이션(Operation) :** 

그래프 상의 노드는 오퍼레이션(줄임말 op)으로 불립니다. 오퍼레이션은 하나 이상의 텐서를 받을 수 있습니다. 오퍼레이션은 계산을 수행하고, 결과를 하나 이상의 텐서로 반환할 수 있습니다.

**텐서(Tensor) :** 

내부적으로 모든 데이터는 텐서를 통해 표현됩니다. 텐서는 일종의 다차원 배열인데, 그래프 내의 오퍼레이션 간에는 텐서만이 전달됩니다. (Caffe의 Blob과 유사합니다.)

**세션(Session) :** 

그래프를 실행하기 위해서는 세션 객체가 필요합니다. 세션은 오퍼레이션의 실행 환경을 캡슐화한 것입니다.

**변수(Variables) :**

변수는 그래프의 실행시, 패러미터를 저장하고 갱신하는데 사용됩니다. 메모리 상에서 텐서를 저장하는 버퍼 역할을 합니다.

# Basic Usage

1. 컴퓨테이션을 그래프로 나타낸다.
2. 그래프는 Sessions 위에서 실행한다.
3. 데이터는 Tensor로 나타낸다.
4. 상태를 Variables 와 함께 유지한다.
5. 데이터를 쓰기 (연산하기) 위해 feeds와 fetches를 사용한다.

# The computation graph

텐서플로는 그래프를 조립하는 construction phase와 그래프의 ops를 실행하기 위해 세션을 사용하는 execution phase로 구성된다.

예를 들면, 인공신경망을 학습하는 그래프는 construction phase에서 생성되고, 이것이 반복적으로 실행되면서 실질적으로 트레이닝 ops가 실행되는 것은 execution phase에서 일어난다.

In [1]:
import tensorflow as tf
graph = tf.get_default_graph()
operations = graph.get_operations()
print operations

[]


In [2]:
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])

product = tf.matmul(matrix1, matrix2)

# 이렇게 쓰면, 바로 계산되는 것이 아니라, ops constructor가 default graph에 nodes를 추가한다. 
# 대부분의 경우에 이 디폴트 그래프로 전체 플로우를 나타낼 수 있다. 여러개의 그래프가 필요할 경우 Graph class 도큐먼트를 참조하자.

operations = graph.get_operations()
print operations

# Build the graph

일단 **import** 하면 내부적으로 **default_graph_stack**에 **default Graph**가 생긴다. 
***tf.get_default_graph()***명령어로 접근 가능
이 graph에 저장된 operation을 확인해 보면 []비어 있는 것을 알 수 있다.
상수를 하나 선언 하면 아래처럼 주소가 기록된다.
이러한 의미는 **operation이 리스트 형태**로 들어가 있는 것이다.
TensorFlow는 내부적으로 ***protocol buffer***를 이용한다.
어떤 Google 스타일의 JSON이라고 생각하면 쉽다. 위에 출력도 JSON 스럽기 때문이다.


In [3]:
operations[0].node_def

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
        dim {
          size: 1
        }
        dim {
          size: 2
        }
      }
      tensor_content: "\000\000@@\000\000@@"
    }
  }
}

In [4]:
print matrix1
print matrix2
print product

Tensor("Const:0", shape=(1, 2), dtype=float32)
Tensor("Const_1:0", shape=(2, 1), dtype=float32)
Tensor("MatMul:0", shape=(1, 1), dtype=float32)


# Launching the graph in a session

이제, 디폴트 그래프는 3개의 노드를 가진다: 2개의 constant() ops 와 하나의 matmul() op. 실제로 결과를 얻기 위해서는 이 그래프를 세션에 올려야 한다. 

세션에서 모든 연산은 패러렐하게 작동한다. 세션 constructor에 파라메터를 넘기지 않았기 때문에 디폴트 그래프로 작동한다. 세션 API는 Session class 도큐먼트를 참고하자.

작업이 끝나면 리소스를 해방하기 위해 close()가 필요하다. 파이썬에서 이러한 구조는 with 를 통해 간단하게 쓸 수 있다

# Interactive Usage

이와 같이, 텐서플로에는 그래프를 만들고 이를 세션에 올려서 Session.run() 을 통해 실행시킨다. 이러한 과정은 IPython 등의 interactive python environment에서는 불편할 수 있으므로 이를 위한 환경을 제공한다 - ***InteractiveSession, Tensor.eval(), Operation.run()***

import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

x.initializer.run()

sub = tf.sub(x, a)
print sub.eval()

In [6]:
sess = tf.Session()

# 결과값 'result'는 numpy의 `ndarray`로 출력된다.
result = sess.run(product)
print result

sess.close()

[[ 12.]]
float32


In [7]:
with tf.Session() as sess:
  result = sess.run(product)
  print result

[[ 12.]]


# Variables

변수는 그래프의 실행 사이에 상태를 유지한다. Variables 참고. 간단한 카운터 예제를 보자:

아래 과정에서 sess.run(update)는 tf.assign(state, new_value) 라는 op인데, 이 op는 다시 new_value 노드를 호출하고 이는 tf.add(state, one) 이라는 op이다. 그리고 이는 다시 one 노드로 가서 tf.constant(1) 이라는 op를 호출할 것이다. 반면, state는 Variable이기 때문에 초기 생성자 tf.Variable(0, name=”counter”) 까지 올라가지 않고 state에 저장된 값을 사용한다.


In [4]:
import tensorflow as tf

# 변수를 0으로 초기화
state = tf.Variable(0, name="counter")

# state에 1을 더할 오퍼레이션 생성
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# 그래프는 처음에 변수를 초기화해야 합니다. 아래 함수를 통해 init 오퍼레이션을 만듭니다.   
init_op = tf.initialize_all_variables()

# 그래프를 띄우고 오퍼레이션들을 실행
with tf.Session() as sess:
  # 초기화 오퍼레이션 실행
  sess.run(init_op)
  # state의 초기 값을 출력
  print(sess.run(state))
  # state를 갱신하는 오퍼레이션을 실행하고, state를 출력
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))

0
1
2
3


# Fetches

연산의 결과를 fetch 하는 (가져오는) 함수는, 이미 위에서도 많이 썼지만, run()이다. 지금까지는 싱글 노드만을 fetch했지만 여러개도 할 수 있다.

# Feeds

fetch가 값을 가져오는 개념이었다면, feed는 값을 넣는 개념이다. 지금까지 값을 넣기 위해 Constant와 Variable을 사용했다. 이 외에 텐서를 직접적으로 그래프의 op에 넣는 방법도 있다.
아래와 같이 run()에서 feed를 넣어주면 넣은 tensor가 임시로 op (위 예제에서는 tf.placeholder()) 를 대체한다. 일반적으로 “feed” 를 하기 위해서 명시적으로 op를 생성하고자 할 때 tf.placeholder()를 사용한다. 위 예제 맨 위의 주석과 같이, constant와 같이 다른 op 를 대체할 수 있으나 type은 같아야 한다 (tf.constant(1) 을 하면 에러가 난다).

placeholder()의 경우 feed를 넣지 않으면 에러가 난다.

In [11]:
input1 = tf.constant(7.)
input2 = tf.placeholder(tf.float32, [1])
output = tf.mul(input1, input2)

with tf.Session() as sess:
  print sess.run([output], feed_dict={input2:[2.]})

[array([ 14.], dtype=float32)]
